In [69]:
pip install pymorphy3

In [70]:
pip install transformers torch sentencepiece

In [71]:
import pymorphy3
import re
from transformers import pipeline
import nltk
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

morph = pymorphy3.MorphAnalyzer()
nltk.download('punkt')

nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [72]:
# 1. Лемматизация и стемминг с помощью pymorphy3
def lemmatize_text(text):
    tokens = re.findall(r'\w+', text.lower())  # # первый аргумент из модуя "re" означает что мы выбираем целые слова, и преорбразуем их все в нижний регистр
    lemmatized_tokens = [morph.parse(token)[0].normal_form for token in tokens]
    return " ".join(lemmatized_tokens)

In [73]:
stemmer = SnowballStemmer("russian")
def stem_nltk(text):
  words = text.split()  # Простейшая токенизация
  res = []
  for word in words:
    res.append(stemmer.stem(word))
  return " ".join(res) #Вернем текст

In [74]:
text_rus = "Это пример предложения на русском языке, демонстрирующий удаление стоп-слов."
#1. Удаляем стоп слова из оригинального текста
#2. Лемматизируем и стеммим текст
lemmatized_text = lemmatize_text(text_rus)
stemmed_text = stem_nltk(text_rus)


print("Исходныйt:", text_rus)
print("Лемитизированный:", lemmatized_text)
print("Стемминговый текст:", stemmed_text)

Исходныйt: Это пример предложения на русском языке, демонстрирующий удаление стоп-слов.
Лемитизированный: это пример предложение на русский язык демонстрировать удаление стоп слово
Стемминговый текст: эт пример предложен на русск языке, демонстрир удален стоп-слов.


In [75]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-ru-en")

text_eng = translator(text_rus)[0]['translation_text']
print(text_eng)

Device set to use cpu


This is an example of a proposal in Russian showing the deletion of the stop words.


In [76]:
stemmer = PorterStemmer()  # Указываем язык!

def stem_nltk(text):
  words = text.split()  # Простейшая токенизация
  res = []
  for word in words:
    res.append(stemmer.stem(word))
  return " ".join(res)



In [77]:

#2. Лемматизируем и стеммим только отфильтрованный текст
lemmatized_text_eng = lemmatize_text(text_eng)
stemmed_text_eng = stem_nltk(text_eng)


print("Исходный текст:", text_eng)
print("Лемитизированный текст:", lemmatized_text_eng)
print("Стеминговый текст:", stemmed_text_eng)

Исходный текст: This is an example of a proposal in Russian showing the deletion of the stop words.
Лемитизированный текст: this is an example of a proposal in russian showing the deletion of the stop words
Стеминговый текст: thi is an exampl of a propos in russian show the delet of the stop words.


In [78]:
# 2. Функция для токенизации всех символов из ASCII
def tokenize(text_eng: str) -> list[str]:
    tokens = []
    for char in text_eng:
        if 0 <= ord(char) <= 127:  #check if char is ascii char
            tokens.append(char)
    return tokens
tok = tokenize(text_eng)
print(tok)

['T', 'h', 'i', 's', ' ', 'i', 's', ' ', 'a', 'n', ' ', 'e', 'x', 'a', 'm', 'p', 'l', 'e', ' ', 'o', 'f', ' ', 'a', ' ', 'p', 'r', 'o', 'p', 'o', 's', 'a', 'l', ' ', 'i', 'n', ' ', 'R', 'u', 's', 's', 'i', 'a', 'n', ' ', 's', 'h', 'o', 'w', 'i', 'n', 'g', ' ', 't', 'h', 'e', ' ', 'd', 'e', 'l', 'e', 't', 'i', 'o', 'n', ' ', 'o', 'f', ' ', 't', 'h', 'e', ' ', 's', 't', 'o', 'p', ' ', 'w', 'o', 'r', 'd', 's', '.']


In [79]:
def vectorize(tokens: list[str]) -> list[int]:
    """Vectorizes a list of ASCII characters to a list of their ASCII codes."""
    vectors = []
    for token in tokens:
        if len(token) == 1 and 0 <= ord(token) <= 127:
            vectors.append(ord(token))
    return vectors

In [80]:

# 4. Токенизация и векторизация текста после лемматизации и стемминга
lemmatized_tokens = lemmatize_text(text_eng)
stemmed_tokens = stem_nltk(text_eng)


tokenized_lemmatized_text = tokenize("".join(lemmatized_tokens))
vectorized_lemmatized_text = vectorize(tokenized_lemmatized_text)


tokenized_stemmed_text = tokenize("".join(stemmed_tokens))
vectorized_stemmed_text = vectorize(tokenized_stemmed_text)

print("\nAfter Lemmatization:")
print("Tokenized (first 5):", tokenized_lemmatized_text[:30])
print("Vectorized (first 5):", vectorized_lemmatized_text[:30])

print("\nAfter Stemming:")
print("Tokenized (first 5):", tokenized_stemmed_text[:30])
print("Vectorized (first 5):", vectorized_stemmed_text[:30])

# Example Usage with "Big text"



After Lemmatization:
Tokenized (first 5): ['t', 'h', 'i', 's', ' ', 'i', 's', ' ', 'a', 'n', ' ', 'e', 'x', 'a', 'm', 'p', 'l', 'e', ' ', 'o', 'f', ' ', 'a', ' ', 'p', 'r', 'o', 'p', 'o', 's']
Vectorized (first 5): [116, 104, 105, 115, 32, 105, 115, 32, 97, 110, 32, 101, 120, 97, 109, 112, 108, 101, 32, 111, 102, 32, 97, 32, 112, 114, 111, 112, 111, 115]

After Stemming:
Tokenized (first 5): ['t', 'h', 'i', ' ', 'i', 's', ' ', 'a', 'n', ' ', 'e', 'x', 'a', 'm', 'p', 'l', ' ', 'o', 'f', ' ', 'a', ' ', 'p', 'r', 'o', 'p', 'o', 's', ' ', 'i']
Vectorized (first 5): [116, 104, 105, 32, 105, 115, 32, 97, 110, 32, 101, 120, 97, 109, 112, 108, 32, 111, 102, 32, 97, 32, 112, 114, 111, 112, 111, 115, 32, 105]
